In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

from pandas.tseries.offsets import CustomBusinessDay

In [3]:
def org_columns(df):
    '''
    Organiza os dataframes, arruma nomes das colunas, remove linhas
    e colunas que possuam apenas NAs
    '''
    tickers = df.columns.str.extract('(.*)$')[0]
    df.columns = tickers
    df[df.index.str.contains("Jan|Feb|Apr|May|Jul|Aug|Oct|Nov")]=np.NaN
    df = df.fillna(method='ffill', limit=2)
    df.columns.name = "ticker"
    return df

In [17]:
# 1) Return on Asset (ROA%) for latest FY is > 0 = Score 1
# 1) Return on Assets - roa
roa_ati = pd.read_excel('../../data/fscore/roa_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
roa_can = pd.read_excel('../../data/fscore/roa_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
roa_can.drop(["Jan-86", "Feb-86"], inplace=True)
roa = pd.concat([roa_ati,roa_can],axis=1,sort=False)
roa = org_columns(roa)
roa = roa[roa.index.str.contains("Dec")]

In [5]:
# 2) Operating Cash Flow for latest FY is > Operating Cash Flow from 1 year ago ( FY-1) = Score 1
# 2) Operating Cash Flow - ocf
# Fica como lição que sempre que for fazer uma função para abrir um arquivo, dá uma olhada a olho nu no arquivo, nos
# limites, etc. As soluções serão ad hoc.
ocf_ati = pd.read_excel('../../data/fscore/cash_op_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ocf_can = pd.read_excel('../../data/fscore/cash_op_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ocf_ati.drop(["Jan-86", "Feb-86"], inplace=True)
ocf = pd.concat([ocf_ati,ocf_can],axis=1,sort=False)
ocf = org_columns(ocf)
ocf = ocf[ocf.index.str.contains("Dec")]

In [ ]:
# 3) Return on Assets (ROA%) for latest FY is > FY prior = Score 1
# 3) Change in Return on Assets
# Basta usar dados anteriores

In [ ]:
# 4) Operating Cash Flow for latest FY is > Net Income (before extraordinary items) for latest FY = Score 1
# 4) Accruals - accs
# Na verdade, Accruals pode ser estimado como
# Accruals = NI - Operating Cash Flows
# Um accrual negativo seria bom, devido à quality of earnings
# Na verdade não vou usar accruals, mas apenas testar se Operating Cash Flow > NI
# Operating Cash Flow já temos falta NI

In [6]:
# Net Income - ni
ni_ati = pd.read_excel('../../data/fscore/net_income_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ni_can = pd.read_excel('../../data/fscore/net_income_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ni_can.drop(["Jan-86", "Feb-86"], inplace=True)
ni = pd.concat([ni_ati,ni_can],axis=1,sort=False)
ni = org_columns(ni)
ni = ni[ni.index.str.contains("Dec")]

In [ ]:
# 5) Long-term Debt to asset growth over 1 year is < 0 = Score 1
# Precisarei de Long-term Debt e de Total Assets

In [7]:
# Long-term Debt - ltd
ltd_ati = pd.read_excel('../../data/fscore/total_debt_LT_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ltd_can = pd.read_excel('../../data/fscore/total_debt_LT_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ltd_can.drop(["Jan-86", "Feb-86"], inplace=True)
ltd = pd.concat([ltd_ati,ltd_can],axis=1,sort=False)
ltd = org_columns(ltd)
ltd = ltd[ltd.index.str.contains("Dec")]

In [8]:
# Total Assets - ta
ta_ati = pd.read_excel('../../data/fscore/total_assets_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ta_can = pd.read_excel('../../data/fscore/total_assets_mensal_cancelandas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ta_can.drop(["Jan-86", "Feb-86"], inplace=True)
ta = pd.concat([ta_ati,ta_can],axis=1,sort=False)
ta = org_columns(ta)
ta = ta[ta.index.str.contains("Dec")]

In [9]:
# Long-term Debt to asset
ltdta = ltd.div(ta)

In [ ]:
# 6) Current Ratio Growth
# 6) Current Ratio growth over 1 year is > 0 = Score 1

In [10]:
# Current Ratio - cr
cr_ati = pd.read_excel('../../data/fscore/current_ratio_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
cr_can = pd.read_excel('../../data/fscore/current_ratio_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
cr_can.drop(["Jan-86", "Feb-86"], inplace=True)
cr = pd.concat([cr_ati,cr_can],axis=1,sort=False)
cr = org_columns(cr)
cr = cr[cr.index.str.contains("Dec")]

In [11]:
# 7) Number of Shares - ns
# 7) No New Shares Issues in the preceding FY = Score 1
# alguns number of shares tem vírgula, estranho
ns_ati = pd.read_excel('../../data/fscore/number_shares_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ns_can = pd.read_excel('../../data/fscore/number_shares_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ns_can.drop(["Jan-86", "Feb-86"], inplace=True)
ns = pd.concat([ns_ati,ns_can],axis=1,sort=False)
ns.columns = ns.columns.str.extract('(.*)$')[0]
ns.columns.name = "ticker"
ns.drop(["Oct-19"], inplace=True)
ns = ns[ns.index.str.contains("Dec")]

In [12]:
# 8) Gross Margin - gm
# 8) Gross Margin growth over 1 year is > 0 = Score 1
gm_ati = pd.read_excel('../../data/fscore/gross_margin_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
gm_can = pd.read_excel('../../data/fscore/gross_margin_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
gm_can.drop(["Jan-86", "Feb-86"], inplace=True)
gm = pd.concat([gm_ati,gm_can],axis=1,sort=False)
gm = org_columns(gm)
gm = gm[gm.index.str.contains("Dec")]

In [13]:
# 9) Asset Turnover - at
# 9) Asset Turnover growth over 1 year is > 0 = Score 1
at_ati = pd.read_excel('../../data/fscore/asset_turnover_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
at_can = pd.read_excel('../../data/fscore/asset_turnover_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
at_can.drop(["Jan-86", "Feb-86"], inplace=True)
at = pd.concat([at_ati,at_can],axis=1,sort=False)
at = org_columns(at)
at = at[at.index.str.contains("Dec")]

In [128]:
# Volume Mensal (in thousands) - vm
vm_ati = pd.read_excel('../../data/economatica_1/volume_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
vm_can = pd.read_excel('../../data/economatica_1/volume_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
vm_ati.drop(["Jan-86", "Feb-86"], inplace=True)
vm_can.drop(["Jan-86", "Feb-86"], inplace=True)
vm = pd.concat([vm_ati,vm_can],axis=1,sort=False)
vm.columns = vm.columns.str.extract('(.*)$')[0]
vm.columns.name = "ticker"
vm.drop(["Oct-19"], inplace=True)

In [141]:
# Market Value of Equity - mve
# mve tem 404 rows enquanto normalmente as outras tem 403, o que causa problema na hora de somar, para ver qual a diferença
# mve.index.difference(nibd.index)
# mve tem outubro de 2019 a mais
mve_ati = pd.read_excel('../../data/economatica_1/valor_mercado_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
mve_can = pd.read_excel('../../data/economatica_1/valor_mercado_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
mve_can.drop(["Jan-86", "Feb-86"], inplace=True)
mve = pd.concat([mve_ati,mve_can],axis=1,sort=False)
mve.drop(["Oct-19"], inplace=True)
mve = org_columns(mve)
mve = mve[mve.index.str.contains("Dec")]

In [137]:
mve

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Mar-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
May-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jun-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jul-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aug-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sep-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oct-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nov-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
print("total: ", ltd.size)
print("ltd:",ltd.isnull().sum().sum())
print("ta:",ta.isnull().sum().sum())
print("ltdta:",ltdta.isnull().sum().sum())

total:  42702
ltd: 25009
ta: 23405
ltdta: 25010


In [ ]:
# Uma pergunta que devemos nos fazer é se a carteira vai ser renovada de trimestre a trimestre ou mês a mês
# Vou fazer rodando só no final de ano

In [ ]:
# O Objetivo final é chegar num dataframe com papéis n coluna, datas nas linhas e scores como dados. Para isso vou criar
# dataframes de data X papel para cada um dos fatores e depois somar tudo

In [40]:
# Fator 1 - ROA
# factor_1 = roa # na verdade cuidar pois essa é uma shallow copy, compartilha memória
factor_1 = roa.copy()
positives = roa > 0
negatives = roa < 0
factor_1[positives] = 1
factor_1[negatives] = 0
factor_1.tail()

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,0.0,1.0,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,0.0,NaN
Dec-15,0.0,1.0,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,0.0,NaN
Dec-16,0.0,1.0,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN
Dec-17,0.0,1.0,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
Dec-18,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN


In [42]:
# Fator 2 - Delta Operating Cash Flow
factor_2 = ocf.shift(1, axis = 0) - ocf # delta operating cash flow
positives = factor_2 > 0
negatives = factor_2 < 0
factor_2[positives] = 1
factor_2[negatives] = 0
factor_2.tail()

ticker,QVUM3B,APPA3,APPA4,ABCB3,ABCB11,ABYA3,AVIL3,AVIL4,AELP3,AESL3,...,WEGE3,MWET3,MWET4,WHRL3,WHRL4,WSON33,WIZS3,WLMM3,WLMM4,YDUQ3
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,1.0,0.0
Dec-15,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
Dec-16,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
Dec-17,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Dec-18,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# Fator 3 - Delta ROA
factor_3 = roa.shift(1, axis = 0) - roa
positives = factor_3 > 0
negatives = factor_3 < 0
factor_3[positives] = 1
factor_3[negatives] = 0
factor_3.tail()

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
Dec-15,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN
Dec-16,1.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN
Dec-17,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
Dec-18,0.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [43]:
# Fator 4 - Operating Cash Flow > Net Income
factor_4 = ocf-ni
positives = factor_4 > 0
negatives = factor_4 < 0
factor_4[positives] = 1
factor_4[negatives] = 0
factor_4.tail()

ticker,AALR3,ABCB11,ABCB3,ABCB4,ABEV3,ABRE11,ABYA3,ACES3,ACES4,ACGU3,...,WISA4,WIZS3,WLMM3,WLMM4,WMBY3,WMBY4,WSON33,WWOW3,YDUQ3,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN
Dec-15,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN
Dec-16,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN
Dec-17,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN
Dec-18,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN


In [44]:
# Fator 5 - Long-term Debt to asset growth
factor_5 = ltdta.shift(1, axis = 0) - ltdta
positives = factor_5 > 0
negatives = factor_5 < 0
factor_5[positives] = 1
factor_5[negatives] = 0
factor_5.tail()

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,0.0,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
Dec-15,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,0.0,0.0,NaN
Dec-16,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN
Dec-17,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
Dec-18,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN


In [45]:
# Fator 6 - Current Ratio growth
factor_6 = cr.shift(1, axis = 0) - cr
positives = factor_6 > 0
negatives = factor_6 < 0
factor_6[positives] = 1
factor_6[negatives] = 0
factor_6.tail()

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
Dec-15,NaN,0.0,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,0.0,NaN
Dec-16,NaN,0.0,NaN,NaN,0.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN
Dec-17,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
Dec-18,NaN,0.0,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN


In [64]:
# Fator 7 - Delta number of shares
factor_7 = ns.shift(1, axis = 0) - ns
zeros = factor_7 == 0
positives = factor_7 > 0 # assim é muito melhor que != pois mantém os NAs
negatives = factor_7 < 0
factor_7[zeros] = 1
factor_7[positives] = 0 
factor_7[negatives] = 0

factor_7.tail()

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,1.0
Dec-15,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
Dec-16,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
Dec-17,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
Dec-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0


In [49]:
# Fator 8 - Gross Margin Growth
factor_8 = gm.shift(1, axis = 0) - gm
positives = factor_8 > 0
negatives = factor_8 < 0
factor_8[positives] = 1
factor_8[negatives] = 0
factor_8.tail()

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
Dec-15,NaN,1.0,NaN,NaN,NaN,0.0,0.0,0.0,1.0,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN
Dec-16,NaN,0.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
Dec-17,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
Dec-18,NaN,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [50]:
# Fator 9 - Asset Turnover Growth
factor_9 = at.shift(1, axis = 0) - at
positives = factor_9 > 0
negatives = factor_9 < 0
factor_9[positives] = 1
factor_9[negatives] = 0
factor_9.tail()

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN
Dec-15,0.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,0.0,NaN
Dec-16,0.0,1.0,NaN,NaN,0.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
Dec-17,0.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
Dec-18,0.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [65]:
print("factor_1: ", factor_1.shape, factor_1.isnull().sum().sum())
print("factor_2: ", factor_2.shape, factor_2.isnull().sum().sum())
print("factor_3: ", factor_3.shape, factor_3.isnull().sum().sum())
print("factor_4: ", factor_4.shape, factor_4.isnull().sum().sum())
print("factor_5: ", factor_5.shape, factor_5.isnull().sum().sum())
print("factor_6: ", factor_6.shape, factor_6.isnull().sum().sum())
print("factor_7: ", factor_7.shape, factor_7.isnull().sum().sum())
print("factor_8: ", factor_8.shape, factor_8.isnull().sum().sum())
print("factor_9: ", factor_9.shape, factor_9.isnull().sum().sum())

factor_1:  (33, 1294) 23632
factor_2:  (33, 1294) 35248
factor_3:  (33, 1294) 25000
factor_4:  (33, 1294) 34252
factor_5:  (33, 1294) 26261
factor_6:  (33, 1294) 24934
factor_7:  (33, 1294) 14138
factor_8:  (33, 1294) 26209
factor_9:  (33, 1294) 25000


In [78]:
# Dá pau pois tem um monte de NAs
# factors = factor_1 + factor_2 + factor_3 + factor_4 + factor_5 + factor_6 + factor_7 + factor_8 + factor_9
factors = factor_1.add(factor_2, fill_value=0).add(factor_3, fill_value=0).add(factor_4, fill_value=0).add(factor_5, fill_value=0).add(factor_6, fill_value=0).add(factor_7, fill_value=0).add(factor_8, fill_value=0).add(factor_9, fill_value=0)
factors.tail()

ticker,AALR3,ABCB11,ABCB3,ABCB4,ABEV3,ABRE11,ABYA3,ACES3,ACES4,ACGU3,...,WISA4,WIZS3,WLMM3,WLMM4,WMBY3,WMBY4,WSON33,WWOW3,YDUQ3,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-14,NaN,5.0,5.0,5.0,5.0,4.0,1.0,1.0,1.0,1.0,...,1.0,2.0,6.0,6.0,4.0,4.0,6.0,1.0,4.0,1.0
Dec-15,1.0,3.0,3.0,3.0,6.0,7.0,1.0,1.0,1.0,1.0,...,1.0,4.0,6.0,6.0,6.0,6.0,6.0,6.0,5.0,1.0
Dec-16,5.0,3.0,3.0,3.0,6.0,4.0,1.0,1.0,1.0,1.0,...,1.0,4.0,6.0,6.0,1.0,1.0,7.0,0.0,6.0,1.0
Dec-17,6.0,6.0,6.0,6.0,5.0,3.0,1.0,1.0,1.0,1.0,...,1.0,6.0,3.0,3.0,1.0,1.0,6.0,1.0,3.0,1.0
Dec-18,5.0,5.0,5.0,5.0,6.0,5.0,1.0,1.0,1.0,1.0,...,1.0,3.0,5.0,5.0,1.0,1.0,7.0,1.0,4.0,1.0


In [ ]:
# Para contar o número de trues e falses usar value_counts(). O problema é que essa é uma função das Series.
#goodones.ABEV3.value_counts()
#goodones.loc["Dec-16"].value_counts()

#for index, row in goodones.iterrows():
#    print(index,row.value_counts().to_string())

In [168]:
good_ones = factors>=7
big_ones = mve>5000000

candidates = good_ones & big_ones

In [169]:
candidates.sum(axis=1).tail(15)

Date
Dec-04     0
Dec-05     2
Dec-06    13
Dec-07    21
Dec-08     7
Dec-09    40
Dec-10    22
Dec-11    28
Dec-12    30
Dec-13    30
Dec-14    13
Dec-15    13
Dec-16    16
Dec-17    16
Dec-18     9
dtype: int64

In [162]:
good_ones.sum(axis=1).tail(15)

Date
Dec-04     0
Dec-05     9
Dec-06    12
Dec-07    17
Dec-08    10
Dec-09    52
Dec-10    35
Dec-11    42
Dec-12    37
Dec-13    31
Dec-14    41
Dec-15    52
Dec-16    29
Dec-17    34
Dec-18    19
dtype: int64

In [143]:
print(bigones.shape)
print(goodones.shape)

(33, 1294)
(33, 1294)


In [145]:
bigones

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-86,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-87,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-88,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-89,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-90,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-91,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-92,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
Dec-93,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,True,False,False,False,False
Dec-94,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,True,False,False,False,True


In [146]:
goodones

ticker,AALR3,ABCB11,ABCB3,ABCB4,ABEV3,ABRE11,ABYA3,ACES3,ACES4,ACGU3,...,WISA4,WIZS3,WLMM3,WLMM4,WMBY3,WMBY4,WSON33,WWOW3,YDUQ3,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Dec-86,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-87,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-88,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-89,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-90,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-91,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-92,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-93,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Dec-94,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
